# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [1]:
%pip install lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pythonw.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the URL using the `requests` library.

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)

Use Beautiful Soup to parse this string into a tree called `soup`

In [4]:
# YOUR CODE HERE
soup = BeautifulSoup(response.content, "html.parser")

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to an element that looks like

```
<table class="______" style="_______">
```

There are many `<table>` tags on the page.  One option you have is to find them all, and then manually figure out which one in this list is the table you care about.

In [5]:
all_tables = len(soup.find_all("table"))
all_tables

10



The other option is to try to use attributes like `class=` and/or `style=` to narrow down the list.

In [6]:
# this didnt work
len(soup.find_all("table",
                  attrs={
                      "class":"sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center jquery-tablesorter",
                      "style":"text-align:right"}
                  ))

0

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [7]:
table = soup.find_all("table")[2]

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [8]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [9]:
rows = []
for row in table.find_all("tr")[2:]:
  cells = row.find_all("td")
  cells

  city = cells[0].text.strip()
  city

  state = cells[1].text.strip()
  state

  pop = cells[2].text.strip()
  pop

  area = cells[5].text.strip()
  area

  rows.append({"city": city, "state": state, "population (2024 estimate)": pop, "2020 land area (sq mi)": area})

In [10]:
cities_df = pd.DataFrame(rows)
cities_df

,city,state,population (2024 estimate),2020 land area (sq mi)
0,New York[c],NY,"8,478,072",300.5
1,Los Angeles,CA,"3,878,704",469.5
2,Chicago,IL,"2,721,308",227.7
3,Houston,TX,"2,390,125",640.4
4,Phoenix,AZ,"1,673,164",518.0
...,...,...,...,...
341,Deltona,FL,"100,513",37.3
342,Federal Way,WA,"100,252",22.3
343,San Angelo,TX,"100,159",59.7
344,Tracy,CA,"100,136",25.9


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [11]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [12]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [13]:
response = requests.get("https://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")
soup = BeautifulSoup(response.content, "html.parser")

In [14]:
courses = soup.find_all("div", class_="courseblock")
courses[0]

<div class="courseblock">
<p class="courseblocktitle"><strong>DATA 100. Data Science for All I.
<span class="courseblockhours">4 units
</span></strong></p><div class="noindent courseextendedwrap">
<p class="noindent">Term Typically Offered: F, W, SP</p><p class="noindent">2020-21 or later catalog: GE Area B4</p><p class="noindent">2019-20 or earlier catalog: GE Area B4</p><p>Prerequisite: <a class="bubblelink code" href="/search/?P=MATH%20115" onclick="return showCourse(this, 'MATH 115');" title="MATH 115">MATH 115</a>, <a class="bubblelink code" href="/search/?P=MATH%20116" onclick="return showCourse(this, 'MATH 116');" title="MATH 116">MATH 116</a>, <a class="bubblelink code" href="/search/?P=MATH%20118" onclick="return showCourse(this, 'MATH 118');" title="MATH 118">MATH 118</a>, or Appropriate Math Placement Level.</p></div>
<div class="courseblockdesc">
<p>Basic approaches for answering questions using data.  Emphasis on working with tabular data in spreadsheet software to provide

In [15]:
course_rows = []

for course in courses:
  name = course.find("p", class_ = "courseblocktitle").text.splitlines()[0]
  #print(name)

  term = course.find("p", class_ = "noindent").text.split(": ")[1]
  #print(term)

  course_rows.append({"Course Name & Number": name, "Term Typically Offered": term})

In [16]:
courses_df = pd.DataFrame(course_rows)
courses_df

,Course Name & Number,Term Typically Offered
0,DATA 100. Data Science for All I.,"F, W, SP"
1,DATA 301. Introduction to Data Science.,"F, W, SP"
2,DATA 401. Data Science Process and Ethics.,F
3,DATA 402. Mathematical Foundations of Data Sci...,F
4,DATA 403. Data Science Projects Laboratory.,F
...,...,...
69,STAT 551. Statistical Learning with R.,F
70,STAT 566. Graduate Consulting Practicum.,SP
71,STAT 570. Selected Advanced Topics.,TBD
72,STAT 590. Graduate Seminar in Statistics.,"F, W, SP"


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag